In [6]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-11-20
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-11-20
@Title : To fetch twitter data from Twitter API

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-11-20\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-11-20\n@Title : To fetch twitter data from Twitter API\n\n'

In [7]:
import boto3
import json
import tweepy
import uuid
import os

In [8]:
BEARER_KEY = os.environ.get('bearer_token')
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')


In [9]:
kinesis_stream_name = 'kds-twitter-sda'
twitter_filter_tag = '#covid lang:en'


class StreamingTweets(tweepy.StreamingClient):
    def on_tweet(self, status):
        pass
        
    
    def on_data(self, data):
        """

        Description:
            To decode data fetched from Twitter API and convert into dictionary.
        Parameter:
            self,
            json data
        Return:
            No values returned.

        """
        status = json.loads(data.decode("utf-8"))
        #print(json.dumps(status, indent=4, sort_keys=True))
        data = {
            'tweet_id': status['data']['id'],
            'tweet_text': status['data']['text'],
            'user_id': status['includes']['users'][0]['id'],
            'user_name': status['includes']['users'][0]['name'],
            'user_username': status['includes']['users'][0]['username']
        }
        #print(json.dumps(data, indent=4, sort_keys=True))
        
        response = kinesis_client.put_record(
            StreamName=kinesis_stream_name,
            Data=json.dumps(data),
            PartitionKey=partition_key)

        print('Status: ' +
              json.dumps(response['ResponseMetadata']['HTTPStatusCode']))
        
        
    def on_error(self, status):
        """

        Description:
            To print status of HTTP connection.
        Parameter:
            int status
        Return:
            No values returned.

        """
        print(status)

def checkIfRuleExists(stream):
    """

    Description:
        To check if there are no hashtag queries i.e. 'rules' in the tweepy Streaming Client.
        If so, delete them all via rule IDs.
    Parameter:
        tweepy.StreamingClient stream
    Return:
        No values returned.

    """
    resp= stream.get_rules()
    if (resp.data != None):
        for rule in resp.data:
            stream.delete_rules(rule.id)


In [10]:
session = boto3.Session()
kinesis_client = session.client('kinesis')
partition_key = str(uuid.uuid4())

stream = StreamingTweets(BEARER_KEY)

checkIfRuleExists(stream)

stream.add_rules(tweepy.StreamRule(twitter_filter_tag))
stream.filter(expansions=['author_id', 'in_reply_to_user_id', 'geo.place_id'])

Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200


KeyboardInterrupt: 